In [1]:
import cv2
import os

def apply_data_augmentation(image_path, save_dir):
    image = cv2.imread(image_path)
    file_name = image_path.split('/')[-1]
    label = file_name.split('\\')[-1].split('_')[0]

    flipped_image = cv2.flip(image, 1)
    
    save_path = os.path.join(save_dir, '{}_flipped_'.format(label) + os.path.basename(image_path))
    cv2.imwrite(save_path, flipped_image)


    for rotation_angle in [45, 90, 135, 180, 225, 270, 315]:
        rotation_matrix = cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), rotation_angle, 1)
        rotated_image = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))
        save_path = os.path.join(save_dir, '{}_rotated{}_'.format(label, rotation_angle) + os.path.basename(image_path))
        cv2.imwrite(save_path, rotated_image)

In [3]:
dataset_dir = r'C:\Users\AHB\Desktop\me'
save_dir = r'C:\Users\AHB\Desktop\me'

for image_file in os.listdir(dataset_dir):
    image_path = os.path.join(dataset_dir, image_file)
    apply_data_augmentation(image_path, save_dir)

In [4]:
import glob

def extract_photo_links(folder_path):
    photo_links = []
    photo_labels = []

    file_pattern = folder_path + '/*.jpg'  

    for file_path in glob.glob(file_pattern):
        file_name = file_path.split('/')[-1]  
        label = file_name.split('\\')[-1].split('_')[0]  

        if label == '1':
            photo_labels.append('Amirhossein')
        elif label == '2':
            photo_labels.append('Morteza')
        else:
            photo_labels.append(label)
        
        photo_links.append(file_name)
    
    print('photo_Links Len: {}'.format(len(photo_links)))
    print('photo_Labels Len: {}'.format(len(photo_labels)))
    return photo_links

In [5]:
folder_path = r'C:\Users\AHB\Desktop\me'
links = extract_photo_links(folder_path)
print(links)

photo_Links Len: 261
photo_Labels Len: 261
['C:\\Users\\AHB\\Desktop\\me\\1_1.jpg', 'C:\\Users\\AHB\\Desktop\\me\\1_10.jpg', 'C:\\Users\\AHB\\Desktop\\me\\1_11.jpg', 'C:\\Users\\AHB\\Desktop\\me\\1_12.jpg', 'C:\\Users\\AHB\\Desktop\\me\\1_13.jpg', 'C:\\Users\\AHB\\Desktop\\me\\1_14.jpg', 'C:\\Users\\AHB\\Desktop\\me\\1_15.jpg', 'C:\\Users\\AHB\\Desktop\\me\\1_16.jpg', 'C:\\Users\\AHB\\Desktop\\me\\1_17.jpg', 'C:\\Users\\AHB\\Desktop\\me\\1_18.jpg', 'C:\\Users\\AHB\\Desktop\\me\\1_19.jpg', 'C:\\Users\\AHB\\Desktop\\me\\1_2.jpg', 'C:\\Users\\AHB\\Desktop\\me\\1_20.jpg', 'C:\\Users\\AHB\\Desktop\\me\\1_21.jpg', 'C:\\Users\\AHB\\Desktop\\me\\1_3.jpg', 'C:\\Users\\AHB\\Desktop\\me\\1_4.jpg', 'C:\\Users\\AHB\\Desktop\\me\\1_5.jpg', 'C:\\Users\\AHB\\Desktop\\me\\1_6.jpg', 'C:\\Users\\AHB\\Desktop\\me\\1_7.jpg', 'C:\\Users\\AHB\\Desktop\\me\\1_8.jpg', 'C:\\Users\\AHB\\Desktop\\me\\1_9.jpg', 'C:\\Users\\AHB\\Desktop\\me\\1_flipped_1_1.jpg', 'C:\\Users\\AHB\\Desktop\\me\\1_flipped_1_10.jpg', 'C:

In [6]:
def detect_faces(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

In [7]:
def load_training_data(folder_path):
    labels = []
    face_images = extract_photo_links(folder_path)
    face_descriptors = []

    for image_path in face_images:
        image = cv2.imread(image_path)
        faces = detect_faces(image)

        if len(faces) != 1:
            continue

        file_name = image_path.split('/')[-1]  
        label = file_name.split('\\')[-1].split('_')[0]

        if label == '1':
            labels.append('Amirhossein')
        elif label == '2':
            labels.append('Morteza')
        else:
            labels.append(label)
        
        (x, y, w, h) = faces[0]
        face_roi = cv2.resize(image[y:y+h, x:x+w], (100, 100))
        face_descriptors.append(face_roi.flatten())
        
    print(labels)

    return face_descriptors, labels

In [8]:
from sklearn.svm import OneClassSVM, SVC


def train_one_class_svm(face_descriptors):
    one_class_svm = OneClassSVM(kernel='rbf')
    one_class_svm.fit(face_descriptors)
    return one_class_svm


def train_svm(face_descriptors, labels):
    svm_classifier = SVC(kernel='rbf')
    svm_classifier.fit(face_descriptors, labels)
    return svm_classifier

In [9]:
def authenticate_face(face_image, svm_classifier):
    faces = detect_faces(face_image)
    if len(faces) != 1:
        return False

    (x, y, w, h) = faces[0]
    face_roi = cv2.resize(face_image[y:y+h, x:x+w], (100, 100))
    face_descriptor = face_roi.flatten()
    prediction = svm_classifier.predict([face_descriptor])

    return prediction[0] if prediction else None

In [10]:
if __name__ == '__main__':

    face_descriptors, labels = load_training_data(r'C:\Users\AHB\Desktop\me'  )
    if not face_descriptors:
        print('No face images were loaded for training.')
        exit()


    svm_classifier = train_svm(face_descriptors, labels)
    oneclass_classifier = train_one_class_svm(face_descriptors)

    cap = cv2.VideoCapture(0)
    cv2.namedWindow('Face Authentication')

    while True:
        ret, frame = cap.read()
        if not ret:
            print('Failed to capture webcam frame.')
            break

        result = authenticate_face(frame, svm_classifier)
        resultp = authenticate_face(frame, oneclass_classifier)

        if result:
            cv2.rectangle(frame, (10, 10), (250, 60), (0, 255, 0), cv2.FILLED)
            cv2.putText(frame, '{} Authenticated: {}'.format(resultp, result), (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
        else:
            cv2.rectangle(frame, (10, 10), (320, 60), (0, 0, 255), cv2.FILLED)
            cv2.putText(frame, 'Authentication Failed', (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)


        cv2.imshow('Face Authentication', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


photo_Links Len: 261
photo_Labels Len: 261
['Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Amirhossein', 'Morteza', 'Morteza', 'Morteza', 'Morteza', 'Morteza', 'Morteza', 'Morteza', 'Morteza', 'Morteza', 'Morteza', 'Morteza', 'Morteza', 'Morteza', 'Morteza', 'Morteza', 'Morteza', 'Morteza', 'Morteza', 'Morteza